In [18]:
import pandas as pd
import sqlite3
import numpy as np
from datetime import datetime
import os

pd.set_option('display.max_columns', 1000)

In [19]:
data_path = os.path.join(os.getcwd(), 'data')
data_files = os.listdir(data_path)

### Merge fbref files into single dataset

In [20]:
fbref_teams_df = pd.read_csv('data/fbref_team_season_stats_standard.csv', sep=';', index_col=0)[['league', 'season', 'team']].drop_duplicates()

for f in [_ for _ in data_files if _.startswith('fbref')]:
    fbref_df = pd.read_csv(os.path.join(data_path, f), sep=';', index_col=0)
    try:
        fbref_df = fbref_df.drop(columns=['url', '#_Pl'])
    except:
        pass
    duplicated_cols = [col for col in fbref_df.columns if col in fbref_teams_df.columns and col not in ['team', 'league', 'season']]
    
    if len(duplicated_cols) > 0:
        fbref_df = fbref_df.drop(columns=duplicated_cols)
    
    fbref_teams_df = fbref_teams_df.merge(fbref_df, how='inner', on=['team', 'league', 'season'])

fbref_teams_df.to_csv(f'data_preprocessed/fbref_team_aggregates.csv', sep=';')

### Divide match_history_data into home and away data

In [21]:
match_history_data = pd.read_csv('data/match_history_data.csv', sep=';', index_col=0).reset_index(drop=True)
match_history_data['Match_Date'] = pd.to_datetime(match_history_data['Match_Date'])

# Filter to played matches
match_history_data = match_history_data[match_history_data['Match_Date'] < datetime.now()]

# Create match_id
match_history_data['match_id'] = np.arange(1, match_history_data.shape[0] + 1)

In [22]:
home_team_match_data = match_history_data[
    [
        'match_id', 'league', 'season', 'Match_Date', 'Home_Team', 'Bet365_home_win_odds', 'Bet365_draw_odds'
    ]
]


away_team_match_data = match_history_data[
    [
        'match_id', 'league', 'season', 'Match_Date', 'Away_Team', 'Bet365_away_win_odds', 'Bet365_draw_odds'
    ]
]

### Merge ELO scores

In [23]:
elo_scores = pd.read_csv('data/elo_scores.csv', sep=';', index_col=0).reset_index(drop=True).drop(columns=['rank', 'country', 'level', 'league'])
elo_scores['from'] = pd.to_datetime(elo_scores['from'])
elo_scores['to'] = pd.to_datetime(elo_scores['to'])
elo_scores = elo_scores.rename(columns={'from': 'start', 'to': 'end'})

In [24]:
# Make the db in memory
conn = sqlite3.connect(':memory:')

# Write the tables
home_team_match_data.to_sql('home_team_match_data', conn, index=False)
away_team_match_data.to_sql('away_team_match_data', conn, index=False)
elo_scores.to_sql('elo_scores', conn, index=False)

sqlcode_home = '''
select A.*, B.elo as elo
from home_team_match_data A
left join elo_scores B 
on A.Home_Team = B.team
and A.Match_Date <= B.end
order by A.match_id, A.Match_Date, B.end
'''
#  and A.Match_Date <= B.end
sqlcode_away = '''
select A.*, B.elo as elo
from away_team_match_data A
left join elo_scores B 
on A.Away_Team = B.team
and A.Match_Date <= B.end
order by A.match_id, A.Match_Date, B.end
'''

home_team_match_elo_data = pd.read_sql_query(sqlcode_home, conn).groupby('match_id').first().reset_index()
away_team_match_elo_data = pd.read_sql_query(sqlcode_away, conn).groupby('match_id').first().reset_index()

### Merge fbref_team_aggregates and fifa data to home and away data

In [25]:
fifa_data = pd.read_csv('data/FIFA_data.csv', sep=';', index_col=0)

In [26]:
# Merge data for home teams
home_team_model_data = home_team_match_elo_data.merge(
    fbref_teams_df, left_on=["season", "Home_Team", "league"], right_on=["season", "team", "league"], how="left"
).merge(
    fifa_data, left_on=["season", "Home_Team"], right_on=["season", "Club"], how="left"
)

# Merge data for away teams
away_team_model_data = away_team_match_elo_data.merge(
    fbref_teams_df, left_on=["season", "Away_Team", "league"], right_on=["season", "team", "league"], how="left"
).merge(
    fifa_data, left_on=["season", "Away_Team"], right_on=["season", "Club"], how="left"
)

### Save model data

In [27]:
home_team_model_data = home_team_model_data.drop(columns=['league', 'season', 'Match_Date', 'Home_Team', 'team', 'Club'])
home_team_model_data = home_team_model_data[home_team_model_data["match_id"].isin(away_team_model_data["match_id"])].sort_values("match_id").reset_index(drop=True)
home_team_model_data.to_csv("data_preprocessed/home_team_model_data.csv", sep=";")

away_team_model_data = away_team_model_data.drop(columns=['league', 'season', 'Match_Date', 'Away_Team', 'team', 'Club']).sort_values("match_id")
away_team_model_data = away_team_model_data[away_team_model_data["match_id"].isin(home_team_model_data["match_id"])].sort_values("match_id").reset_index(drop=True)
away_team_model_data.to_csv("data_preprocessed/away_team_model_data.csv", sep=";")

In [38]:
# Home to away
home_to_away_model_data = pd.concat([home_team_model_data[["match_id", "Bet365_home_win_odds", "Bet365_draw_odds"]], home_team_model_data.drop(columns=["match_id", "Bet365_home_win_odds", "Bet365_draw_odds"]) / away_team_model_data.drop(columns=["match_id", "Bet365_away_win_odds", "Bet365_draw_odds"])], axis=1)
home_to_away_model_data["match_id"] = home_to_away_model_data["match_id"].map(lambda x: "home_" + str(x))

# Away to home
away_to_home_model_data = pd.concat([away_team_model_data[["match_id", "Bet365_away_win_odds", "Bet365_draw_odds"]], away_team_model_data.drop(columns=["match_id", "Bet365_away_win_odds", "Bet365_draw_odds"]) / home_team_model_data.drop(columns=["match_id", "Bet365_home_win_odds", "Bet365_draw_odds"])], axis=1)
away_to_home_model_data["match_id"] = away_to_home_model_data["match_id"].map(lambda x: "away_" + str(x))

In [40]:
home_to_away_model_data.to_csv("data_preprocessed/home_to_away_model_data.csv", sep=";")
away_to_home_model_data.to_csv("data_preprocessed/away_to_home_model_data.csv", sep=";")

### Save target data

In [41]:
match_history_data.to_csv("data_preprocessed/sample_matches.csv", sep=';')

In [42]:
home_match_history_data = match_history_data[['match_id', 'Full_Time_Home_Team_Goals', 'Home_Team_Shots_on_Target', 'Result_Home_Team_Win', 'Result_Draw']]
home_match_history_data["match_id"] = home_match_history_data["match_id"].map(lambda x: "home_" + str(x))

away_match_history_data = match_history_data[['match_id', 'Full_Time_Away_Team_Goals', 'Away_Team_Shots_on_Target', 'Result_Away_Team_Win', 'Result_Draw']]
away_match_history_data["match_id"] = away_match_history_data["match_id"].map(lambda x: "away_" + str(x))

C:\Users\kubag\AppData\Local\Temp/ipykernel_8292/2408851681.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_match_history_data["match_id"] = home_match_history_data["match_id"].map(lambda x: "home_" + str(x))
C:\Users\kubag\AppData\Local\Temp/ipykernel_8292/2408851681.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_match_history_data["match_id"] = away_match_history_data["match_id"].map(lambda x: "away_" + str(x))


In [43]:
home_match_history_data.to_csv("data_preprocessed/home_match_history_data.csv", sep=';')
away_match_history_data.to_csv("data_preprocessed/away_match_history_data.csv", sep=';')